In [1]:
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# def clean_lyrics(string,xWords):
#     for word in xWords:
#         string = string.replace(word, '')
#     return string

# xWords = ['[Intro]','[Verse 1]','[Verse 2]', '[Pre-Chorus]', '[Chorus]', '[Post-Chorus]']
# clean_lyrics = clean_lyrics(lyrics.text, xWords)
# print(clean_lyrics, '\n')

# unnecessary_words = ['a', 'an', 'and', 'the', 'of']
# print(' '.join(i for i in clean_lyrics.split() if i not in unnecessary_words))